# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# check current working directory
print(os.getcwd())

# get file path of source data
filepath = os.getcwd() + '/event_data'
print(filepath)

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
file_path_list

print(f'Total number of event file is: {len(file_path_list)}')

/Users/mr_yue/Desktop/DATA_ENGINEER/DataEngineerProject/Data Modeling with Cassandra
/Users/mr_yue/Desktop/DATA_ENGINEER/DataEngineerProject/Data Modeling with Cassandra/event_data
Total number of event file is: 30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_row_list = []

# for every filepath in the file path list 
for f in file_path_list:
    
    # reading csv file 
    with open(f, 'r', encoding='utf8',newline='') as csvfile:
        # create a csv reader object
        csvreader = csv.reader(csvfile)
        next(csvreader)
        
        # extracting each data row one by one and append it
        for row in csvreader:
            full_data_row_list.append(row)

# get total number of rows 
print(f'Total number of rows is {len(full_data_row_list)}')
# list of event data rows
full_data_row_list[:20]

Total number of rows is 8056


[['Harmonia',
  'Logged In',
  'Ryan',
  'M',
  '0',
  'Smith',
  '655.77751',
  'free',
  'San Jose-Sunnyvale-Santa Clara, CA',
  'PUT',
  'NextSong',
  '1.54102E+12',
  '583',
  'Sehr kosmisch',
  '200',
  '1.54224E+12',
  '26'],
 ['The Prodigy',
  'Logged In',
  'Ryan',
  'M',
  '1',
  'Smith',
  '260.07465',
  'free',
  'San Jose-Sunnyvale-Santa Clara, CA',
  'PUT',
  'NextSong',
  '1.54102E+12',
  '583',
  'The Big Gundown',
  '200',
  '1.54224E+12',
  '26'],
 ['Train',
  'Logged In',
  'Ryan',
  'M',
  '2',
  'Smith',
  '205.45261',
  'free',
  'San Jose-Sunnyvale-Santa Clara, CA',
  'PUT',
  'NextSong',
  '1.54102E+12',
  '583',
  'Marry Me',
  '200',
  '1.54224E+12',
  '26'],
 ['',
  'Logged In',
  'Wyatt',
  'M',
  '0',
  'Scott',
  '',
  'free',
  'Eureka-Arcata-Fortuna, CA',
  'GET',
  'Home',
  '1.54087E+12',
  '563',
  '',
  '200',
  '1.54225E+12',
  '9'],
 ['',
  'Logged In',
  'Austin',
  'M',
  '0',
  'Rosales',
  '',
  'free',
  'New York-Newark-Jersey City, NY-NJ-PA',

In [4]:
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding='utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_row_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6],
                         row[7], row[8], row[12], row[13], row[16]))
            

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('The total number of rows in  processed event datafile is: ' + str(sum(1 for line in f)))

The total number of rows in  processed event datafile is: 6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


In [9]:
# Based on query, we should create table first.

query_1_table = """
    CREATE TABLE IF NOT EXISTS songs_with_session_item (
        sessionId INT,
        itemInSession INT,
        artist TEXT,
        songTitle TEXT,
        songLength FLOAT,
        PRIMARY KEY (sessionId, itemInSession)  
    );
"""

try:
    session.execute(query_1_table)
except Exception as e:
    print(e)

    

In [10]:
# insert values into table based on source code
query_1_insert = """
    INSERT INTO songs_with_session_item (sessionId, itemInSession, artist, songTitle, songLength)
    VALUES (%s, %s, %s, %s, %s);
"""
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        artist_name, first_name, gender, itemInSession, last_name, length, level, location, sessionId, song, userId = line
        session.execute(query_1_insert, (int(sessionId), int(itemInSession), artist_name, song, float(length)))

        

In [11]:
# generate query to meet requirement
query_1 = """
    select 
        artist,
        songTitle,
        songLength
    from songs_with_session_item
    where sessionId = 338
    and itemInSession = 4
"""
results = session.execute(query_1)

for row in results:
    print(f'Artist: {row.artist}, Song Title: {row.songtitle}, Song length: {row.songlength}')

Artist: Faithless, Song Title: Music Matters (Mark Knight Dub), Song length: 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
# create table for query 2
query_2_table = """
    CREATE TABLE IF NOT EXISTS song_with_user_session (
        userId INT,
        sessionId INT,
        itemInSession INT,
        artist TEXT,
        song TEXT,
        firstName TEXT,
        lastName TEXT,
        PRIMARY KEY ((userId, sessionId), itemInSession)
        );
"""

try:
    session.execute(query_2_table)
except Exception as e:
    print(e)

In [13]:
# insert into query2 table values
query_2_insert = """
    INSERT INTO song_with_user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
"""
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        artist_name, first_name, gender, itemInSession, last_name, length, level, location, sessionId, song, userId = line
        session.execute(query_2_insert, (int(userId), int(sessionId), int(itemInSession), artist_name, song, first_name, last_name))

In [14]:
# generate query to meet query2 requirement
query_2 = """
    select
        artist,
        song,
        firstName,
        lastName
    from song_with_user_session
    where userId = 10
    and sessionId = 182;
"""
results_2 = session.execute(query_2)
for row in results_2:
    print(f"Artist: {row.artist}, Song: {row.song}, first_name: {row.firstname}, last_name: {row.lastname}")

Artist: Down To The Bone, Song: Keep On Keepin' On, first_name: Sylvie, last_name: Cruz
Artist: Three Drives, Song: Greece 2000, first_name: Sylvie, last_name: Cruz
Artist: Sebastien Tellier, Song: Kilometer, first_name: Sylvie, last_name: Cruz
Artist: Lonnie Gordon, Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), first_name: Sylvie, last_name: Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [15]:
# create query3 table
query_3_table = """
    CREATE TABLE IF NOT EXISTS user_with_song (
        song TEXT,
        userId INT,
        firstName TEXT,
        lastName TEXT,
        PRIMARY KEY (song, userId)
   );
"""
try:
    session.execute(query_3_table)
except Exception as e:
    print(e)

In [16]:
# insert values into query3 table
query_3_insert = """
    INSERT INTO user_with_song (song, userId, firstName, lastName)
    VALUES (%s, %s, %s, %s);
"""
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        artist_name, first_name, gender, itemInSession, last_name, length, level, location, sessionId, song, userId = line
        session.execute(query_3_insert, (song, int(userId), first_name, last_name))

In [17]:
# generate query3 to meet requirement
query_3 = """
    select
        firstName,
        lastName
    from user_with_song
    where song = 'All Hands Against His Own';
"""
results_3 = session.execute(query_3)
for row in results_3:
    print(f'First Name: {row.firstname}, Last Name: {row.lastname}')

First Name: Jacqueline, Last Name: Lynch
First Name: Tegan, Last Name: Levine
First Name: Sara, Last Name: Johnson


### Drop the tables before closing out the sessions.

In [18]:
try:
    session.execute("DROP TABLE IF EXISTS songs_with_session_item")
    session.execute("DROP TABLE IF EXISTS song_with_user_session")
    session.execute("DROP TABLE IF EXISTS user_with_song")
except Exception as e:
    print(e)

 ### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()